In [ ]:
!pip install flwr
!pip install torch torchvision
!pip install -U flwr[simulation]
!pip install "ray[default]" --ignore-installed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.1/465.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.12.5
    Uninstalling typer-0.12.5:
      Successfully uninstalled typer-0.12.5
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.1
    Uninstalling cryptography-43.0.1:
      Successfully uninstalled cryptography-43.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This b

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import zipfile
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import ndarrays_to_parameters, Context
from flwr.server import ServerApp, ServerConfig
from flwr.server import ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
import flwr as fl
from flwr.client import NumPyClient
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential,models,layers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.applications.resnet50 import  ResNet50
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import VGG16, EfficientNetB6
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import tensorflow as tf
from typing import Dict, List
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from flwr.common import NDArrays, Scalar
import warnings

warnings.filterwarnings("ignore")

In [ ]:

!unzip "BrainTumor.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Training/glioma/Tr-gl_0712.jpg  
  inflating: Training/glioma/Tr-gl_0713.jpg  
  inflating: Training/glioma/Tr-gl_0714.jpg  
  inflating: Training/glioma/Tr-gl_0715.jpg  
  inflating: Training/glioma/Tr-gl_0716.jpg  
  inflating: Training/glioma/Tr-gl_0717.jpg  
  inflating: Training/glioma/Tr-gl_0718.jpg  
  inflating: Training/glioma/Tr-gl_0719.jpg  
  inflating: Training/glioma/Tr-gl_0720.jpg  
  inflating: Training/glioma/Tr-gl_0721.jpg  
  inflating: Training/glioma/Tr-gl_0722.jpg  
  inflating: Training/glioma/Tr-gl_0723.jpg  
  inflating: Training/glioma/Tr-gl_0724.jpg  
  inflating: Training/glioma/Tr-gl_0725.jpg  
  inflating: Training/glioma/Tr-gl_0726.jpg  
  inflating: Training/glioma/Tr-gl_0727.jpg  
  inflating: Training/glioma/Tr-gl_0728.jpg  
  inflating: Training/glioma/Tr-gl_0729.jpg  
  inflating: Training/glioma/Tr-gl_0730.jpg  
  inflating: Training/glioma/Tr-gl_0731.jpg  
  inflating: Training/glioma/

In [ ]:
train_ds="/content/Training"
test_ds="/content/Testing"


In [ ]:
train_giloma="/content/Training/glioma"
train_meningioma="/content/Training/meningioma"
train_pituitary="/content/Training/pituitary"
train_no_tumor="/content/Training/notumor"

test_giloma="/content/Testing/glioma"
test_meningioma="/content/Testing/meningioma"
test_pituitary="/content/Testing/pituitary"
test_no_tumor="/content/Testing/notumor"

train_sets=[train_giloma,train_meningioma,train_pituitary,train_no_tumor]
test_sets=[test_giloma,test_meningioma,test_pituitary,test_no_tumor]


In [ ]:
class_names= os.listdir(train_ds)
n_classes = len(class_names)
print(f"Total Number of Classes : {n_classes} \nClass Names : {class_names}")

Total Number of Classes : 4 
Class Names : ['glioma', 'notumor', 'meningioma', 'pituitary']


In [ ]:
batch_size=32
image_size = (224, 224)

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(train_ds,target_size= image_size ,batch_size=batch_size,shuffle=True,class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_ds,target_size= image_size ,batch_size=batch_size,shuffle=True,class_mode='categorical')

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base of VGG16 to retain pre-trained weights
vgg_base.trainable = False

# Add custom layers on top of the VGG16 base
x = Flatten()(vgg_base.output)  # Flatten the output from the conv base
x = Dense(512, activation='relu')(x)  # Add a fully connected layer
x = Dense(4, activation='softmax')(x)  # 4 output classes

model = Model(inputs=vgg_base.input, outputs=x)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 27,562,308 (105.14 MB)

 Trainable params: 12,847,620 (49.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
batch_size=32
image_size = (224, 224)

train_no_tumor_datagen = ImageDataGenerator(rescale=1./255)
test_no_tumor_datagen = ImageDataGenerator(rescale=1./255)

train_no_tumor_generator = train_no_tumor_datagen.flow_from_directory(
    '/content/Training',
    target_size= image_size ,batch_size=batch_size,shuffle=True,class_mode='categorical'
)

test_no_tumor_generator = test_no_tumor_datagen.flow_from_directory(
    '/content/Testing',target_size= image_size ,batch_size=batch_size,shuffle=True,class_mode='categorical')

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




In [ ]:
history = model.fit(train_no_tumor_generator, epochs= 1, verbose= 1, validation_data=test_no_tumor_generator , shuffle= False)

Epoch 1/10
150/179 ━━━━━━━━━━━━━━━━━━━━ 9:35 20s/step - accuracy: 0.7423 - loss: 1.2951

In [ ]:
def get_weights(model):
    """Retrieve model weights in a format compatible with Flower."""
    return model.get_weights()

def set_weights(model, weights):
    """Set model weights using Flower parameters."""
    model.set_weights(weights)


In [ ]:
# Federated Learning Functions
def set_weights(model, parameters):
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)

def get_weights(model):
    ndarrays = [val.cpu().numpy() for _, val in model.state_dict().items()]
    return ndarrays

In [ ]:
# FlowerClient class for Federated Learning
class FlowerClient(NumPyClient):
    def __init__(self, model, trainset, testset):
        self.model = model
        self.trainset = trainset
        self.testset = testset

    # Train the model
    def fit(self, parameters: NDArrays, config: Dict[str, Scalar]):
        set_weights(self.model, parameters)
        self.model.fit(self.trainset, epochs=1)  # Train for one epoch per round
        return get_weights(self.model), len(self.trainset), {}

    # Evaluate the model
    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]):
        set_weights(self.model, parameters)
        loss, accuracy = self.model.evaluate(self.testset)
        return loss, len(self.testset), {"accuracy": accuracy}

In [ ]:
def evaluate_model(net, testset,config):
    # Custom evaluation for the four tumor classes
    loss, accuracy = net.evaluate(test_generator)
    _, accuracy_glioma = net.evaluate(test_giloma)
    _, accuracy_meningioma = net.evaluate(test_meningioma)
    _, accuracy_pituitary = net.evaluate(test_pituitary)
    _, accuracy_no_tumor = net.evaluate(test_no_tumor)

    print(f"Test accuracy (all): {accuracy:.4f}")
    print(f"Test accuracy (glioma): {accuracy_glioma:.4f}")
    print(f"Test accuracy (meningioma): {accuracy_meningioma:.4f}")
    print(f"Test accuracy (pituitary): {accuracy_pituitary:.4f}")
    print(f"Test accuracy (no tumor): {accuracy_no_tumor:.4f}")

In [ ]:
def client_fn(context: Context) -> NumPyClient:
    # Load VGG16 with custom layers
    model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model.trainable = False

    # Add custom layers
    x = Flatten()(model.output)
    x = Dense(512, activation='relu')(x)
    output_layer = Dense(4, activation='softmax')(x)

    # Final model
    model = Model(inputs=model.input, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Partitioning dataset
    partition_id = int(context.node_config["partition-id"])
    client_train = train_generator
    client_test = test_generator

    return FlowerClient(model, client_train, client_test)


In [ ]:
def server_fn(context: Context):
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.0,
        initial_parameters=ndarrays_to_parameters(initial_parameters),
        evaluate_fn=evaluate,
    )
    config=ServerConfig(num_rounds=3)
    return ServerAppComponents(
        strategy=strategy,
        config=config,
    )

In [ ]:
server = ServerApp(server_fn=server_fn)
run_simulation(server_app=server, client_app=client_fn, num_supernodes=5)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
ERROR :     ServerApp thread raised an exception: 'int' object has no attribute 'evaluate'
ERROR :     Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/run_simulation.py", line 336, in server_th_with_start_checks
    run_server_app(
  File "/usr/local/lib/python3.10/dist-packages/flwr/server/run_serverapp.py", line 88, in run
    server_app(driver=driver, context=context)
  File "/usr/local/lib/python3.10/dist-packages/flwr/server/server_app.py", line 120, in __call__
    start_driver(
  File "/usr/local/lib/python3.10/dist-packages/flwr/server/compat/app.py", line 87, in start_driver
    hist = run_fl(
  File "/usr/local/lib/python3.10/dist-packages/flwr/server/server.py", line 492, in run_fl
    hist, e

RuntimeError: Exception in ServerApp thread

In [ ]:
import os
from collections import OrderedDict
import numpy as np
import torch
from flwr.client import NumPyClient
from flwr.server.strategy import FedAvg
from flwr.server.app import start_server
from flwr.simulation.app import start_simulation
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data paths for training and testing
train_ds="/content/Training"
test_ds="/content/Testing"

## 4 parts for training
train_giloma="/content/Training/glioma"
train_meningioma="/content/Training/meningioma"
train_pituitary="/content/Training/pituitary"
train_no_tumor="/content/Training/notumor"

## 4 parts for testing
test_giloma="/content/Testing/glioma"
test_meningioma="/content/Testing/meningioma"
test_pituitary="/content/Testing/pituitary"
test_no_tumor="/content/Testing/notumor"

train_sets=[train_giloma,train_meningioma,train_pituitary,train_no_tumor]
test_sets=[test_giloma,test_meningioma,test_pituitary,test_no_tumor]

## Below is the model I want to use

# VGG16 model as the base
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_base.trainable = False

# Custom layers
x = Flatten()(vgg_base.output)
x = Dense(512, activation='relu')(x)
x = Dense(4, activation='softmax')(x)

# Compile model
model = Model(inputs=vgg_base.input, outputs=x)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()





## Image preprocessing for Training

batch_size=32
image_size = (224, 224)

train_no_tumor_datagen = ImageDataGenerator(rescale=1./255)
test_no_tumor_datagen = ImageDataGenerator(rescale=1./255)

train_no_tumor_generator = train_no_tumor_datagen.flow_from_directory(
    '/content/Training',
    target_size= image_size ,batch_size=batch_size,shuffle=True,class_mode='categorical'
)

test_no_tumor_generator = test_no_tumor_datagen.flow_from_directory(
    '/content/Testing',target_size= image_size ,batch_size=batch_size,shuffle=True,class_mode='categorical')









train_sets=[train_giloma, train_meningioma, train_pituitary, train_no_tumor]
test_sets=[test_giloma, test_meningioma, test_pituitary, test_no_tumor]

class_names= os.listdir(train_ds)
n_classes = len(class_names)
print(f"Total Number of Classes : {n_classes} \nClass Names : {class_names}")

# Model configuration
batch_size=32
image_size = (224, 224)

# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_ds, target_size=image_size, batch_size=batch_size, shuffle=True, class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_ds, target_size=image_size, batch_size=batch_size, shuffle=True, class_mode='categorical'
)

# VGG16 model as the base
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_base.trainable = False

# Custom layers
x = Flatten()(vgg_base.output)
x = Dense(512, activation='relu')(x)
x = Dense(4, activation='softmax')(x)

# Compile model
model = Model(inputs=vgg_base.input, outputs=x)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

def get_weights(model):
    """Retrieve model weights in a format compatible with Flower."""
    weights = model.get_weights()
    if len(weights) == 0:
        raise ValueError("No weights found. Ensure the model is initialized properly.")
    return weights


def set_weights(model, weights):
    """Set model weights using Flower parameters."""
    if len(weights) != len(model.get_weights()):
        raise ValueError(f"Received {len(weights)} weights, but the model expects {len(model.get_weights())} weights.")
    model.set_weights(weights)




class FlowerClient(NumPyClient):
    def __init__(self, model, trainset, testset):
        self.model = model
        self.trainset = trainset
        self.testset = testset

    def fit(self, parameters, config):
        """Train the model."""
        set_weights(self.model, parameters)  # Set weights from the server

        # Unpack dataset if necessary (for example, if using `tf.data.Dataset`)
        if isinstance(self.trainset, tuple):
            train_data, train_labels = self.trainset
        else:
            train_data, train_labels = zip(*self.trainset)

        self.model.fit(train_data, train_labels, epochs=1, batch_size=32)  # Customize the batch size if needed

        return get_weights(self.model), len(train_data), {}

    def evaluate(self, parameters, config):
        """Evaluate the model."""
        set_weights(self.model, parameters)  # Set the weights from the server

        # Unpack dataset if necessary
        if isinstance(self.testset, tuple):
            test_data, test_labels = self.testset
        else:
            test_data, test_labels = zip(*self.testset)

        loss, accuracy = self.model.evaluate(test_data, test_labels)

        return loss, len(test_data), {"accuracy": accuracy}


# Evaluate model for each tumor class
def evaluate_model(net, test_generator, test_giloma, test_meningioma, test_pituitary, test_no_tumor):
    loss, accuracy = net.evaluate(test_generator)
    _, accuracy_glioma = net.evaluate(test_giloma)
    _, accuracy_meningioma = net.evaluate(test_meningioma)
    _, accuracy_pituitary = net.evaluate(test_pituitary)
    _, accuracy_no_tumor = net.evaluate(test_no_tumor)

    print(f"Test accuracy (all): {accuracy:.4f}")
    print(f"Test accuracy (glioma): {accuracy_glioma:.4f}")
    print(f"Test accuracy (meningioma): {accuracy_meningioma:.4f}")
    print(f"Test accuracy (pituitary): {accuracy_pituitary:.4f}")
    print(f"Test accuracy (no tumor): {accuracy_no_tumor:.4f}")


def client_fn(context):
    net = model  # Use the VGG16-based model
    partition_id = int(context.node_config["partition-id"])
    # Use modulo operator (%) to ensure partition_id is within the valid range
    client_train = train_sets[partition_id % len(train_sets)]
    client_test = test_sets[partition_id % len(test_sets)]
    return FlowerClient(net, client_train, client_test)


# Server function
def server_fn(context):
    net = model  # Use the VGG16-based model
    params = get_weights(net)
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.0,
        initial_parameters=params,
        evaluate_fn=evaluate_model
    )
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)

# Simulation setup
simulation = start_simulation( # run_simulation was renamed to start_simulation
    client_fn=client_fn,
    num_clients=5, # num_supernodes was renamed to num_clients in flwr 1.0.0 and this is the number of clients in the simulation

)


Total Number of Classes : 4 
Class Names : ['glioma', 'notumor', 'meningioma', 'pituitary']
Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 27,562,308 (105.14 MB)

 Trainable params: 12,847,620 (49.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

INFO :      Starting Flower simulation, config: num_rounds=1, no round_timeout
2024-09-24 18:38:23,599	INFO worker.py:1786 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 7835598030.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3917799014.0, 'GPU': 1.0, 'accelerator_type:T4': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=16061) 2024-09-24 18:38:28.697883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory f

In [ ]:
import os
import numpy as np
import torch
from flwr.client import NumPyClient
from flwr.server.strategy import FedAvg
from flwr.simulation.app import start_simulation
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data paths for training and testing
train_ds = "/content/Training"
test_ds = "/content/Testing"

# Initialize class names and number of classes
class_names = os.listdir(train_ds)
n_classes = len(class_names)
print(f"Total Number of Classes: {n_classes}\nClass Names: {class_names}")

# Model configuration
batch_size = 32
image_size = (224, 224)

# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_ds, target_size=image_size, batch_size=batch_size, shuffle=True, class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_ds, target_size=image_size, batch_size=batch_size, shuffle=False, class_mode='categorical'
)

# VGG16 model as the base
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_base.trainable = False

# Custom layers
x = Flatten()(vgg_base.output)
x = Dense(512, activation='relu')(x)
x = Dense(n_classes, activation='softmax')(x)

# Compile model
model = Model(inputs=vgg_base.input, outputs=x)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

def get_weights(model):
    """Retrieve model weights in a format compatible with Flower."""
    weights = model.get_weights()
    if len(weights) == 0:
        raise ValueError("No weights found. Ensure the model is initialized properly.")
    return weights

def set_weights(model, weights):
    """Set model weights using Flower parameters."""
    if len(weights) != len(model.get_weights()):
        raise ValueError(f"Received {len(weights)} weights, but the model expects {len(model.get_weights())} weights.")
    model.set_weights(weights)

class FlowerClient(NumPyClient):
    def __init__(self, model, trainset, testset):
        self.model = model
        self.trainset = trainset
        self.testset = testset

    def fit(self, parameters, config):
        """Train the model."""
        set_weights(self.model, parameters)  # Set weights from the server
        self.model.fit(self.trainset, epochs=1, batch_size=batch_size)  # Update as necessary
        return get_weights(self.model), len(self.trainset), {}

    def evaluate(self, parameters, config):
        """Evaluate the model."""
        set_weights(self.model, parameters)  # Set the weights from the server
        loss, accuracy = self.model.evaluate(self.testset)
        return loss, len(self.testset), {"accuracy": accuracy}

def client_fn(context):
    net = model  # Use the VGG16-based model
    partition_id = int(context.node_config["partition-id"])
    client_train = train_generator  # Use the generator directly
    client_test = test_generator  # Use the generator directly
    return FlowerClient(net, client_train, client_test)

# Simulation setup
simulation = start_simulation(
    client_fn=client_fn,
    num_clients=5,
)


Total Number of Classes: 4
Class Names: ['glioma', 'notumor', 'meningioma', 'pituitary']
Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 27,562,308 (105.14 MB)

 Trainable params: 12,847,620 (49.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

INFO :      Starting Flower simulation, config: num_rounds=1, no round_timeout
2024-09-24 18:46:15,084	INFO worker.py:1786 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 7835993703.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3917996851.0, 'GPU': 1.0, 'accelerator_type:T4': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
ERROR :     Traceback (most recent call last):
  File "python/ray/_raylet.pyx", line 877, in ray._raylet.prepare_args_internal
  File "/usr/local/lib/python3.10/dist-pac

RuntimeError: Simulation crashed.

(pid=18327) 2024-09-24 18:46:20.823717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=18327) 2024-09-24 18:46:20.901418: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=18327) 2024-09-24 18:46:20.921490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import os
from collections import OrderedDict
import numpy as np
import torch
from flwr.client import NumPyClient
from flwr.server.strategy import FedAvg
from flwr.server.app import start_server
from flwr.simulation.app import start_simulation
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data paths for training and testing
train_ds = "/content/Training"
test_ds = "/content/Testing"

class_names = os.listdir(train_ds)
n_classes = len(class_names)
print(f"Total Number of Classes : {n_classes} \nClass Names : {class_names}")

# Model configuration
batch_size = 32
image_size = (224, 224)

def create_model():
    # VGG16 model as the base
    vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vgg_base.trainable = False

    # Custom layers
    x = Flatten()(vgg_base.output)
    x = Dense(512, activation='relu')(x)
    x = Dense(n_classes, activation='softmax')(x)

    # Compile model
    model = Model(inputs=vgg_base.input, outputs=x)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

class FlowerClient(NumPyClient):
    def __init__(self, train_dir, test_dir):
        self.model = create_model()
        self.train_generator = self.get_data_generator(train_dir)
        self.test_generator = self.get_data_generator(test_dir)

    def get_data_generator(self, directory):
        datagen = ImageDataGenerator(rescale=1./255)
        return datagen.flow_from_directory(directory, target_size=image_size, batch_size=batch_size, class_mode='categorical')

    def fit(self, parameters, config):
        """Train the model."""
        set_weights(self.model, parameters)  # Set weights from the server
        self.model.fit(self.train_generator, epochs=1)  # Fit model on the train generator
        return get_weights(self.model), self.train_generator.samples, {}

    def evaluate(self, parameters, config):
        """Evaluate the model."""
        set_weights(self.model, parameters)  # Set the weights from the server
        loss, accuracy = self.model.evaluate(self.test_generator)
        return loss, self.test_generator.samples, {"accuracy": accuracy}

def client_fn(context):
    partition_id = int(context.node_config["partition-id"])
    client_train = os.path.join(train_ds, class_names[partition_id % n_classes])
    client_test = os.path.join(test_ds, class_names[partition_id % n_classes])
    return FlowerClient(client_train, client_test)

# Simulation setup
simulation = start_simulation(
    client_fn=client_fn,
    num_clients=5,
)



Total Number of Classes : 4 
Class Names : ['glioma', 'notumor', 'meningioma', 'pituitary']


INFO :      Starting Flower simulation, config: num_rounds=1, no round_timeout
2024-09-24 18:49:22,561	INFO worker.py:1786 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 7830257664.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3915128832.0, 'GPU': 1.0, 'accelerator_type:T4': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=19435) 2024-09-24 18:49:25.890224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory f

(ClientAppActor pid=19435) Found 0 images belonging to 0 classes.
(ClientAppActor pid=19435) Found 0 images belonging to 0 classes.


(ClientAppActor pid=19435) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(pid=19434) 2024-09-24 18:49:25.914511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=19434) 2024-09-24 18:49:25.949769: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=19434) 2024-09-24 18:49:25.959813: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(ClientAppActor pid=19435) Found 0 images belonging to 0 classes.
(ClientAppActor pid=19435) Found 0 images belonging to 0 classes.


ERROR :     Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 94, in _submit_job
    out_mssg, updated_context = self.actor_pool.get_client_result(
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_actor.py", line 398, in get_client_result
    return self._fetch_future_result(cid)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_actor.py", line 279, in _fetch_future_result
    res_cid, out_mssg, updated_context = ray.get(
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/worker.py", line 2664, in get
    values, debugger_breakpoint = worker.get_obje

(ClientAppActor pid=19434) Found 0 images belonging to 0 classes. [repeated 14x across cluster]


ERROR :     Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 94, in _submit_job
    out_mssg, updated_context = self.actor_pool.get_client_result(
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_actor.py", line 398, in get_client_result
    return self._fetch_future_result(cid)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_actor.py", line 279, in _fetch_future_result
    res_cid, out_mssg, updated_context = ray.get(
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/worker.py", line 2664, in get
    values, debugger_breakpoint = worker.get_obje

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import flwr as fl
from collections import OrderedDict
import numpy as np

# Data paths
train_giloma = "/content/Training/glioma"
train_meningioma = "/content/Training/meningioma"
train_pituitary = "/content/Training/pituitary"
train_no_tumor = "/content/Training/notumor"

test_giloma = "/content/Testing/glioma"
test_meningioma = "/content/Testing/meningioma"
test_pituitary = "/content/Testing/pituitary"
test_no_tumor = "/content/Testing/notumor"

train_sets = [train_giloma, train_meningioma, train_pituitary, train_no_tumor]
test_sets = [test_giloma, test_meningioma, test_pituitary, test_no_tumor]

# VGG16 model setup
def build_model():
    vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vgg_base.trainable = False

    # Custom layers on top
    x = Flatten()(vgg_base.output)
    x = Dense(512, activation='relu')(x)
    x = Dense(4, activation='softmax')(x)

    model = Model(inputs=vgg_base.input, outputs=x)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Image Preprocessing
batch_size = 32
image_size = (224, 224)

def get_data_generators(train_dir, test_dir):
    train_datagen = ImageDataGenerator(rescale=1.0/255)
    test_datagen = ImageDataGenerator(rescale=1.0/255)

    train_generator = train_datagen.flow_from_directory(
        train_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=True)

    test_generator = test_datagen.flow_from_directory(
        test_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

    return train_generator, test_generator

# Federated Client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_generator, test_generator):
        self.model = model
        self.train_generator = train_generator
        self.test_generator = test_generator

    def get_weights(self):
        return self.model.get_weights()

    def set_weights(self, weights):
        self.model.set_weights(weights)

    def fit(self, parameters, config):
        self.set_weights(parameters)
        self.model.fit(self.train_generator, epochs=1, verbose=2)
        return self.get_weights(), len(self.train_generator), {}

    def evaluate(self, parameters, config):
        self.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.test_generator, verbose=2)
        return loss, len(self.test_generator), {"accuracy": accuracy}

# Client function
def client_fn(cid):
    # Partitioning the dataset for different clients
    train_dir = train_sets[int(cid)]
    test_dir = test_sets[int(cid)]

    # Build the model
    model = build_model()

    # Load data
    train_generator, test_generator = get_data_generators(train_dir, test_dir)

    # Return Flower client
    return FlowerClient(model, train_generator, test_generator)

# Server evaluation function
def evaluate(server_round, parameters, config):
    model = build_model()
    model.set_weights(parameters)

    # Load the global test set
    test_generator, _ = get_data_generators("/content/Training", "/content/Testing")

    # Evaluate on global data
    loss, accuracy = model.evaluate(test_generator, verbose=2)
    print(f"Round {server_round} accuracy: {accuracy}")
    return loss, accuracy

# Federated Learning Strategy
# strategy = fl.server.strategy.FedAvg(
#     fraction_fit=1.0,
#     fraction_evaluate=0.0,
#     initial_parameters=None,  # Initial model parameters will be loaded during training
#     evaluate_fn=evaluate,
#     on_fit_config_fn=lambda rnd: {"epoch_global": rnd},
# )

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0, # Changed to 1.0 so that clients can perform evaluation
    initial_parameters=None,  # Initial model parameters will be loaded during training
    evaluate_fn=evaluate,
    on_fit_config_fn=lambda rnd: {"epoch_global": rnd},
)

# Start the simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=4,  # 4 clients (glioma, meningioma, pituitary, no tumor)
    # num_rounds=3,  # Number of rounds
    strategy=strategy,
)


INFO :      Starting Flower simulation, config: num_rounds=1, no round_timeout
2024-09-24 19:02:51,954	INFO worker.py:1786 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'CPU': 2.0, 'object_store_memory': 3914255155.0, 'node:172.28.0.12': 1.0, 'memory': 7828510311.0, 'GPU': 1.0, 'accelerator_type:T4': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=23583) 2024-09-24 19:02:55.782087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory f

(ClientAppActor pid=23585) Found 0 images belonging to 0 classes.
(ClientAppActor pid=23585) Found 0 images belonging to 0 classes.


(ClientAppActor pid=23585) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      Starting evaluation of initial global parameters
ERROR :     You called `set_weights(weights)` on layer 'functional_10' with a weight list of length 0, but the layer was expecting 30 weights.
ERROR :     Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/app.py", line 339, in start_simulation
    hist = run_fl(
  File "/usr/local/lib/python3.10/dist-packages/flwr/server/server.py", line 492, in run_fl
    hist, elapsed_time = server.fit(
  File "/usr/local/lib/python3.10/dist-packages/flwr/server/server.py", line 95, in fit
    res = self.strategy.evaluate(0, parameters=self.parameters)
  File "/usr/local/lib/python3.10/dist-packages/flwr/server/strategy/fedavg.py", line 167, in evaluate
   

RuntimeError: Simulation crashed.

In [ ]:
# prompt: update flwr

!pip install flwr -U


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import flwr as fl
from collections import OrderedDict
from flwr.common import Context

# Data paths
train_giloma = "/content/Training/glioma"
train_meningioma = "/content/Training/meningioma"
train_pituitary = "/content/Training/pituitary"
train_no_tumor = "/content/Training/notumor"

test_giloma = "/content/Testing/glioma"
test_meningioma = "/content/Testing/meningioma"
test_pituitary = "/content/Testing/pituitary"
test_no_tumor = "/content/Testing/notumor"

train_sets = [train_giloma, train_meningioma, train_pituitary, train_no_tumor]
test_sets = [test_giloma, test_meningioma, test_pituitary, test_no_tumor]

# VGG16 model setup
def build_model():
    vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vgg_base.trainable = False

    # Custom layers on top
    x = Flatten()(vgg_base.output)
    x = Dense(512, activation='relu')(x)
    x = Dense(4, activation='softmax')(x)

    model = Model(inputs=vgg_base.input, outputs=x)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Image Preprocessing
batch_size = 32
image_size = (224, 224)

def get_data_generators(train_dir, test_dir):
    train_datagen = ImageDataGenerator(rescale=1.0/255)
    test_datagen = ImageDataGenerator(rescale=1.0/255)

    train_generator = train_datagen.flow_from_directory(
        train_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=True)

    test_generator = test_datagen.flow_from_directory(
        test_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

    return train_generator, test_generator

# Federated Client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_generator, test_generator):
        self.model = model
        self.train_generator = train_generator
        self.test_generator = test_generator

    def get_weights(self):
        return self.model.get_weights()

    def set_weights(self, weights):
        if weights is not None and len(weights) == len(self.model.get_weights()):
            self.model.set_weights(weights)
        else:
            print("Warning: Mismatch in the number of model weights.")

    def fit(self, parameters, config):
        self.set_weights(parameters)
        self.model.fit(self.train_generator, epochs=1, verbose=2)
        return self.get_weights(), len(self.train_generator), {}

    def evaluate(self, parameters, config):
        self.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.test_generator, verbose=2)
        return loss, len(self.test_generator), {"accuracy": accuracy}

# Client function with updated signature
def client_fn(context: Context):
    partition_id = int(context.node_config["partition_id"])  # Use the partition ID from context
    train_dir = train_sets[partition_id]
    test_dir = test_sets[partition_id]

    # Build the model
    model = build_model()

    # Load data
    train_generator, test_generator = get_data_generators(train_dir, test_dir)

    # Return Flower client
    return FlowerClient(model, train_generator, test_generator)


# Server evaluation function
def evaluate(server_round, parameters, config):
    model = build_model()
    model.set_weights(parameters)

    # Load the global test set
    _, test_generator = get_data_generators("/content/Training", "/content/Testing")

    # Evaluate on global data
    loss, accuracy = model.evaluate(test_generator, verbose=2)
    print(f"Round {server_round} accuracy: {accuracy}")
    return loss, accuracy

# Federated Learning Strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.0,
    initial_parameters=None,  # Initial model parameters will be loaded during training
    evaluate_fn=evaluate,
    on_fit_config_fn=lambda rnd: {"epoch_global": rnd},
)

# Start the simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=4,  # 4 clients (glioma, meningioma, pituitary, no tumor)
    strategy=strategy,
)


INFO :      Starting Flower simulation, config: num_rounds=1, no round_timeout
2024-09-24 19:11:22,627	INFO worker.py:1786 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 7827834471.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3913917235.0, 'accelerator_type:T4': 1.0, 'GPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=27031) 2024-09-24 19:11:25.962423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory f

RuntimeError: Simulation crashed.

# **Working**

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import flwr as fl
from collections import OrderedDict
from flwr.common import Context

# Data paths
train_giloma = "/content/Training/glioma"
train_meningioma = "/content/Training/meningioma"
train_pituitary = "/content/Training/pituitary"
train_no_tumor = "/content/Training/notumor"

test_giloma = "/content/Testing/glioma"
test_meningioma = "/content/Testing/meningioma"
test_pituitary = "/content/Testing/pituitary"
test_no_tumor = "/content/Testing/notumor"

train_sets = [train_giloma, train_meningioma, train_pituitary, train_no_tumor]
test_sets = [test_giloma, test_meningioma, test_pituitary, test_no_tumor]

# VGG16 model setup
def build_model():
    vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vgg_base.trainable = False

    # Custom layers on top
    x = Flatten()(vgg_base.output)
    x = Dense(512, activation='relu')(x)
    x = Dense(4, activation='softmax')(x)

    model = Model(inputs=vgg_base.input, outputs=x)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Image Preprocessing
batch_size = 32
image_size = (224, 224)

def get_data_generators(train_dir, test_dir):
    train_datagen = ImageDataGenerator(rescale=1.0/255)
    test_datagen = ImageDataGenerator(rescale=1.0/255)

    train_generator = train_datagen.flow_from_directory(
        train_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=True)

    test_generator = test_datagen.flow_from_directory(
        test_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

    return train_generator, test_generator

# Federated Client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_generator, test_generator):
        self.model = model
        self.train_generator = train_generator
        self.test_generator = test_generator

    def get_weights(self):
        return self.model.get_weights()

    def set_weights(self, weights):
        if weights is not None and len(weights) == len(self.model.get_weights()):
            self.model.set_weights(weights)
        else:
            print("Warning: Mismatch in the number of model weights.")

    def fit(self, parameters, config):
        self.set_weights(parameters)
        self.model.fit(self.train_generator, epochs=1, verbose=2)
        return self.get_weights(), len(self.train_generator), {}

    def evaluate(self, parameters, config):
        self.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.test_generator, verbose=2)
        return loss, len(self.test_generator), {"accuracy": accuracy}

# Client function with updated signature and `partition_id` handling
def client_fn(context: Context):
    try:
        partition_id = int(context.node_config.get("partition_id", 0))  # Use partition 0 as default
    except KeyError:
        print("Warning: `partition_id` not found in context, defaulting to 0")
        partition_id = 0  # Fallback if partition_id is missing

    train_dir = train_sets[partition_id]
    test_dir = test_sets[partition_id]

    # Build the model
    model = build_model()

    # Load data
    train_generator, test_generator = get_data_generators(train_dir, test_dir)

    # Return Flower client
    return FlowerClient(model, train_generator, test_generator)


def evaluate(server_round, parameters, config):
    model = build_model()
    if parameters: # Check if parameters is not empty
        model.set_weights(parameters)

    # Load the global test set
    _, test_generator = get_data_generators("/content/Training", "/content/Testing")

    # Evaluate on global data
    loss, accuracy = model.evaluate(test_generator, verbose=2)
    print(f"Round {server_round} accuracy: {accuracy}")

    # Return loss and metrics as a dictionary
    return loss, {"accuracy": accuracy}

# Federated Learning Strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.0,
    initial_parameters=None,  # Initial model parameters will be loaded during training
    evaluate_fn=evaluate,
    on_fit_config_fn=lambda rnd: {"epoch_global": rnd},
)

# Start the simulation with resource configuration
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=4,  # 4 clients (glioma, meningioma, pituitary, no tumor)  # Number of rounds
    strategy=strategy,
    client_resources={"num_cpus": 2, "num_gpus": 0.5},  # Allocate sufficient resources
)


INFO :      Starting Flower simulation, config: num_rounds=1, no round_timeout
2024-09-24 19:56:00,352	INFO worker.py:1786 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'CPU': 2.0, 'object_store_memory': 3913603891.0, 'node:172.28.0.12': 1.0, 'memory': 7827207783.0, 'GPU': 1.0, 'accelerator_type:T4': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 0.5}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=42415) 2024-09-24 19:56:04.493579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=42415) 2024-09-24 19:56:

(ClientAppActor pid=42415) Found 0 images belonging to 0 classes.
(ClientAppActor pid=42415) Found 0 images belonging to 0 classes.


(ClientAppActor pid=42415) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
41/41 - 7s - 168ms/step - accuracy: 0.2349 - loss: 1.6750
Round 0 accuracy: 0.23493516445159912


INFO :      initial parameters (loss, other metrics): 1.6750290393829346, {'accuracy': 0.23493516445159912}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


...

(ClientAppActor pid=42415) Found 0 images belonging to 0 classes.


ERROR :     Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 94, in _submit_job
    out_mssg, updated_context = self.actor_pool.get_client_result(
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_actor.py", line 398, in get_client_result
    return self._fetch_future_result(cid)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_actor.py", line 279, in _fetch_future_result
    res_cid, out_mssg, updated_context = ray.get(
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/worker.py", line 2664, in get
    values, debugger_breakpoint = worker.get_obje

(ClientAppActor pid=42415) Warning: Mismatch in the number of model weights.
Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
41/41 - 7s - 165ms/step - accuracy: 0.2906 - loss: 1.4800
Round 1 accuracy: 0.29061785340309143


INFO :      fit progress: (1, 1.479960322380066, {'accuracy': 0.29061785340309143}, 9.906129727000007)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 1 round(s) in 9.91s
INFO :      	History (loss, centralized):
INFO :      		round 0: 1.6750290393829346
INFO :      		round 1: 1.479960322380066
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.23493516445159912), (1, 0.29061785340309143)]}
INFO :      


History (loss, centralized):
	round 0: 1.6750290393829346
	round 1: 1.479960322380066
History (metrics, centralized):
{'accuracy': [(0, 0.23493516445159912), (1, 0.29061785340309143)]}

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import flwr as fl
from collections import OrderedDict
from flwr.common import Context

# Data paths
train_giloma = "/content/Training/glioma"
train_meningioma = "/content/Training/meningioma"
train_pituitary = "/content/Training/pituitary"
train_no_tumor = "/content/Training/notumor"

test_giloma = "/content/Testing/glioma"
test_meningioma = "/content/Testing/meningioma"
test_pituitary = "/content/Testing/pituitary"
test_no_tumor = "/content/Testing/notumor"

train_sets = [train_giloma, train_meningioma, train_pituitary, train_no_tumor]
test_sets = [test_giloma, test_meningioma, test_pituitary, test_no_tumor]
train_dir = "/content/Training"
test_dir = "/content/Testing"

# VGG16 model setup
def build_model():
    vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vgg_base.trainable = False

    # Custom layers on top
    x = Flatten()(vgg_base.output)
    x = Dense(512, activation='relu')(x)
    x = Dense(4, activation='softmax')(x)

    model = Model(inputs=vgg_base.input, outputs=x)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Image Preprocessing
batch_size = 32
image_size = (224, 224)

# def get_data_generators(train_dir, test_dir):
#     train_datagen = ImageDataGenerator(rescale=1.0/255)
#     test_datagen = ImageDataGenerator(rescale=1.0/255)

#     train_generator = train_datagen.flow_from_directory(
#         train_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=True)

#     test_generator = test_datagen.flow_from_directory(
#         test_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

#     return train_generator, test_generator


def get_data_generators(train_dir, test_dir):
    train_datagen = ImageDataGenerator(rescale=1.0/255)
    test_datagen = ImageDataGenerator(rescale=1.0/255)

    train_generator = train_datagen.flow_from_directory(
        train_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=True)

    test_generator = test_datagen.flow_from_directory(
        test_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

    if train_generator.samples == 0:
        raise ValueError(f"No images found in the directory: {train_dir}")
    if test_generator.samples == 0:
        raise ValueError(f"No images found in the directory: {test_dir}")

    return train_generator, test_generator



# Federated Client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_generator, test_generator):
        self.model = model
        self.train_generator = train_generator
        self.test_generator = test_generator

    def get_weights(self):
        return self.model.get_weights()

    def set_weights(self, weights):
        if weights is not None and len(weights) == len(self.model.get_weights()):
            self.model.set_weights(weights)
        else:
            print("Warning: Mismatch in the number of model weights.")

    def fit(self, parameters, config):
        self.set_weights(parameters)
        self.model.fit(self.train_generator, epochs=1, verbose=2)
        return self.get_weights(), len(self.train_generator), {}

    def evaluate(self, parameters, config):
        self.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.test_generator, verbose=2)
        return loss, len(self.test_generator), {"accuracy": accuracy}

# Client function with updated signature and `partition_id` handling
# def client_fn(context: Context):
#     try:
#         partition_id = int(context.node_config.get("partition_id", 0))  # Use partition 0 as default
#     except KeyError:
#         print("Warning: `partition_id` not found in context, defaulting to 0")
#         partition_id = 0  # Fallback if partition_id is missing

#     train_dir = train_sets[partition_id]
#     test_dir = test_sets[partition_id]

#     # Build the model
#     model = build_model()

#     # Load data
#     train_generator, test_generator = get_data_generators(train_dir, test_dir)

#     # Return Flower client, converted to the required client format
#     return FlowerClient(model, train_generator, test_generator).to_client()


def client_fn(context: Context):
    try:
        partition_id = int(context.node_config.get("partition_id", 0))  # Use partition 0 as default
    except KeyError:
        print("Warning: `partition_id` not found in context, defaulting to 0")
        partition_id = 0  # Fallback if partition_id is missing

    train_dir = train_sets[partition_id]
    test_dir = test_sets[partition_id]

    # Log directories being loaded
    print(f"Partition ID: {partition_id}")
    print(f"Training directory: {train_dir}")
    print(f"Testing directory: {test_dir}")

    # Build the model
    model = build_model()

    # Load data
    train_generator, test_generator = get_data_generators(train_dir, test_dir)

    # Return Flower client
    return FlowerClient(model, train_generator, test_generator)


# Global Evaluation function
def evaluate(server_round, parameters, config):
    model = build_model()
    if parameters:  # Check if parameters are not empty
        model.set_weights(parameters)

    # Load the global test set
    _, test_generator = get_data_generators("/content/Training", "/content/Testing")

    # Evaluate on global data
    loss, accuracy = model.evaluate(test_generator, verbose=2)
    print(f"Round {server_round} accuracy: {accuracy}")

    # Return loss and metrics as a dictionary
    return loss, {"accuracy": accuracy}

def evaluate_global_model():
    model = build_model()
    # Assuming you have the global parameters
    global_parameters = strategy.evaluate_global_parameters()

    if global_parameters:  # Check if parameters are available
        model.set_weights(global_parameters)

    # Load the global test set
    _, test_generator = get_data_generators("/content/Training", "/content/Testing")

    # Evaluate on global data
    loss, accuracy = model.evaluate(test_generator, verbose=2)
    print(f"Final global model accuracy: {accuracy}")
    return loss, {"accuracy": accuracy}


# Federated Learning Strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,  # Disable client-side evaluation for now
    initial_parameters=None,  # Initial model parameters will be loaded during training
    evaluate_fn=evaluate,  # Global evaluation function
    on_fit_config_fn=lambda rnd: {"epoch_global": rnd},
)

# Start the simulation with resource configuration
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=4,  # 4 clients (glioma, meningioma, pituitary, no tumor)
    config=fl.server.ServerConfig(num_rounds=3), # Number of rounds
    strategy=strategy,
    client_resources={"num_cpus": 2, "num_gpus": 0.5},  # Allocate sufficient resources
)


INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
2024-09-24 20:01:04,111	INFO worker.py:1786 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 7827296256.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3913648128.0, 'GPU': 1.0, 'accelerator_type:T4': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 0.5}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=44102) 2024-09-24 20:01:09.189657: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=44102) 2024-09-24 20:01:

(ClientAppActor pid=44102) Partition ID: 0
(ClientAppActor pid=44102) Training directory: /content/Training/glioma
(ClientAppActor pid=44102) Testing directory: /content/Testing/glioma


(ClientAppActor pid=44102) I0000 00:00:1727208071.759934   44102 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
(ClientAppActor pid=44102) I0000 00:00:1727208071.760314   44102 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
(ClientAppActor pid=44102) I0000 00:00:1727208071.761195   44102 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
(ClientAppActor pid

RuntimeError: Simulation crashed.

(ClientAppActor pid=44102) Found 0 images belonging to 0 classes.
(ClientAppActor pid=44102) Found 0 images belonging to 0 classes.
